# 例: 会话

首先准备一些工具函数，以及资料库创建部分。这一部分和RAG的例子没有区别，因此直接使用相同的代码，不再解释。

In [1]:
import json

from langchain_core.documents import Document
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage


def show_documents(docs: list[Document]):
    from IPython.display import HTML, display

    html = ""
    html += '<ul style="list-style: none;">'
    for doc in docs:
        html += '<li><div style="margin: 15px 0;  box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); transition: 0.3s;">'
        html += f'<pre style="background-color: #eee; font-size: 10px; border: 1px dashed #ccc; padding: 5px;">{json.dumps(doc.metadata, indent=2, ensure_ascii=False)}</pre>'
        html += f'<pre style="background-color: #eff; padding: 5px;">{doc.page_content}</pre>'
        html += "</div></li>"
    html += "</ul>"
    display(HTML(html))


def show_messages(messages: list[BaseMessage]):
    from IPython.display import HTML, display

    html = ""
    html += '<ul style="list-style: none; margin: 5px 0;">'
    for msg in messages:
        html += '<li><div style="margin: 15px 5px;">'
        match msg.type:
            case "ai":
                html += '<div style="text-align: right; font-size: 24px;">🤖</div>'
                html += f'<pre style="background-color: #eff; float: right; padding: 5px; width: fit-content; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); transition: 0.3s;   border-radius: 5px;">{msg.content}</pre>'
            case "human":
                html += '<div style="text-align: left;font-size: 24px;">👨🏻</div>'
                html += f'<pre style="background-color: #ffe; padding: 5px; width: fit-content; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); transition: 0.3s; border-radius: 5px;">{msg.content}</pre>'
            case _:
                html += (
                    f'<div style="text-align: left;font-size: 24px;">{msg.type}</div>'
                )
                html += f'<pre style="background-color: #eee; padding: 5px;width: fit-content; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); transition: 0.3s;">{msg.content}</pre>'
        html += "</div></li>"
    html += "</ul>"
    display(HTML(html))


def show_answer(message: AIMessage):
    from IPython.display import HTML, display

    html = ""
    html += '<div style="background-color: #eee; padding: 5px;">'
    html += f'<div style="font-size: 9px; color: #333;">id={message.id}</div>'
    html += f'<pre style="background-color: transparent; border: 1px dash #ccc; padding: 5px; width: fit-content;">{message.content}</pre>'
    html += f'<pre style="font-size: 9px;">{json.dumps(message.response_metadata, indent=2)}</pre>'
    html += "</div>"
    display(HTML(html))


markdown_document = """
# 六元素について

生活、仕事、教育。日々の暮らしや社会に関わるさまざまなものが、年齢や性別、時間や場所から自由になる世界が実現できる。ITの可能性は、まだまだ広がります。

そんなITの力で私たちが目指す「感動」や「幸せ」は、すべての人がやりたいことに挑戦できる未来、成長できる未来、成功を目指せる未来。だからこそ、あなたが実現したい未来を実現するための
「プラットフォーム」になりたい。

私たち六元素は、「ITプロフェッショナル力」による顧客の課題解決と「社会課題」に挑む自社サービス開発で、顧客と社会に感動と幸せを創造する「チャレンジ・プラットフォームカンパニー」です。

## 経営理念 PHILOSOPHY

顧客へ感動を、社員へ幸せを。

## ミッション MISSION

ITの力で、感動と幸せを創造する。

## ビジョン VISION

「ITプロフェッショナル力」と「社会課題へ挑むベンチャーマインド」で、半歩先の未来価値を創造し続け、感動と幸せを提供する、「チャレンジ・プラットフォーム カンパニー」。

## バリュー VALUE

We are CHONPS（必須元素）

- Challenger やってみよう
- Heart 情熱と心を込めよう
- One team ワンチームになろう
- New technology 先端技術を取り入れよう
- Professional　プロでいよう
- Study 学び続けよう

## 社名由来 ORIGIN OF COMPANY NAME

### 六元素の意味

あらゆる生命は活動を維持するために酸素、炭素、窒素、水素、燐、硫黄という六つの元素が必要だとされています。私たちは、企業を発展させるためにも顧客、従業員、株主、社会、パートナー、ライバルという六つの元素が大切だと考えます。さらに中国では、まだ発見されていない未来の元素のことを「第六元素」とも言います。社名にはこれら二つの意味合いから、「堅実な基礎を築き、未来の価値を追求していく」という意味を込めています。

"""

from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "H1"),
    ("##", "H2"),
    ("###", "H3"),
]

# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=True
)
md_header_splits = markdown_splitter.split_text(markdown_document)

from unstructured.partition.md import partition_md

for doc in md_header_splits:
    doc.page_content = text = "\n".join(
        [str(e) for e in partition_md(text=doc.page_content)]
    )

# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 250
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)

from typing import List

from langchain_community.retrievers import BM25Retriever
from sudachipy import dictionary, tokenizer


class BM25RetrieverWithScores(BM25Retriever):
    def get_relevant_documents_with_scores(self, query):
        processed_query = self.preprocess_func(query)
        scores = self.vectorizer.get_scores(processed_query)
        docs_and_scores = [(doc, score) for doc, score in zip(self.docs, scores)]
        sorted_docs_and_scores = sorted(
            docs_and_scores, key=lambda x: x[1], reverse=True
        )
        return sorted_docs_and_scores[: self.k]

    def _get_relevant_documents(self, query: str) -> List[Document]:
        processed_query = self.preprocess_func(query)
        scores = self.vectorizer.get_scores(processed_query)
        docs_and_scores = [(doc, score) for doc, score in zip(self.docs, scores)]
        docs_and_scores = sorted(docs_and_scores, key=lambda x: x[1], reverse=True)
        threshold = self.metadata.get("score_threshold", 0)
        docs = []
        for doc, score in docs_and_scores:
            doc.metadata["score"] = score
            if score > threshold:
                docs.append(doc)
        return docs[: self.k]


# 默认BM25是按照空格切分句子，这对日语是无效的。
# 这里使用sudachi (from ワークス徳島人工知能NLP研究所) 来分析日语。


def generate_word_ngrams(text, i, j, binary=False):
    """
    文字列を単語に分割し、指定した文字数のn-gramを生成する関数。

    :param text: 文字列データ
    :param i: n-gramの最小文字数
    :param j: n-gramの最大文字数
    :param binary: Trueの場合、重複を削除
    :return: n-gramのリスト
    """

    tokenizer_obj = dictionary.Dictionary(dict="full").create()
    mode = tokenizer.Tokenizer.SplitMode.A
    tokens = tokenizer_obj.tokenize(text, mode)
    words = [token.surface() for token in tokens]

    ngrams = []

    for n in range(i, j + 1):
        for k in range(len(words) - n + 1):
            ngram = tuple(words[k : k + n])
            ngrams.append(ngram)

    if binary:
        ngrams = list(set(ngrams))  # 重複を削除

    return ngrams


def preprocess_func(text: str) -> List[str]:
    return generate_word_ngrams(text, 1, 1, True)


retriever = BM25RetrieverWithScores.from_documents(
    splits, preprocess_func=preprocess_func, metadata={"score_threshold": 1.5}
)

chunks = retriever.invoke("六元素の意味は何ですか。")

show_documents(chunks)

## 1. 记忆

### 4.1 提示文

增加历史记录，作为记忆。

注意：以下两个概念不完全相等。

- 历史：过去发生的事实。例如，日记本。
- 记忆：对过去事实的参照。例如，交谈时大脑中掌握的过去事件。
  - 基于历史的，但：
    - 不完全：记忆是概要的，会忽略细节
    - 不精确：记忆忽略了细节，可能导致歧义
    - 浓缩的：对比历史，信息量是减少的，体积更小

此处，最简单的，直接以历史作为记忆。
- 优点：精确，没有对历史进行压缩
- 缺点：体积大，不适合长会话。

In [2]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("ai", "What can I help you?"),
        ("human", "Please answer my questions using given documents."),
        ("ai", "OK."),
        MessagesPlaceholder("documents"),
        MessagesPlaceholder("history", optional=True),  # 会话历史
        ("human", "{question}"),  # 最新一条消息
    ]
)

prompt = prompt_template.invoke(
    {
        "question": "What is your name?",
        "history": [
            HumanMessage(content="Balabala..."),
            AIMessage(content="Balabala..."),
        ],
        "documents": [HumanMessage(content="Document 1 ..."), AIMessage(content="OK.")],
    }
)

show_messages(prompt.to_messages())

### 4.2 问题上下文改善

查找相关资料时，使用的是当前一次的“问题”。这涉及上下文指代时，很可能无法获取参考资料。

例如，最新的一条消息是“请继续”。此时不能使用“请继续”三个字去查找相关资料，而应该先搞明白用户到底想问什么，再去查找相关资料。即，“请继续”，要翻译成具体的“请继续回答xxx问题”之后，再依此去查找相关资料。

In [3]:
chunks = retriever.invoke("英語で回答してください。")

show_documents(chunks)

上面的例子是没有输出的，因为根本查找不到相似的资料片段。通过如下改造后，就可以顺利查找出参考资料了。

首先，对问题进行上下文的展开。

In [4]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
    "Please use Japanese."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("history"),
        ("human", "{question}"),
    ]
)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

contextualize_chain = contextualize_q_prompt | llm

res = contextualize_chain.invoke(
    {
        "question": "英語で回答してください。",
        "history": [
            HumanMessage(content="六元素は何ですか。"),
            AIMessage(content="六元素は日本のIT企業です。"),
        ],
    }
)

show_answer(res)

如下的结果处理与RAG例子相同，直接使用即可。

In [5]:
from langchain_core.documents import Document
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.runnables import RunnableLambda


# 将Documents一览变换成Message列表
def convert(docs: list[Document]) -> list[BaseMessage]:
    if len(docs) == 0:
        return [
            HumanMessage(
                content="Sorry. There is no documents found for my question. Please answer my questions without reference documents."
            ),
            AIMessage(
                content="OK. I will answer your questions by original training data."
            ),
        ]
    else:
        messages: list[BaseMessage] = []
        messages.extend(
            [
                HumanMessage(content="The documents will send to you one by one."),
                AIMessage(content="OK. Please show them one by one."),
            ]
        )
        for i, doc in enumerate(docs):
            meta = "\n".join(f"- {k}: {v}" for k, v in doc.metadata.items())
            messages.extend(
                [
                    HumanMessage(
                        content=f"[Document {i+1}]\nMetadata: \n{meta} \n\nContent: \n {doc.page_content}"
                    ),
                    AIMessage(content="OK."),
                ]
            )
        messages.extend(
            [
                HumanMessage(content=f"OK. All {len(docs)} documents are sent to you."),
                AIMessage(
                    content=f"Thank you. I will answer your questions based on those {len(docs)} documents."
                ),
            ]
        )
        return messages


documents_parser = RunnableLambda(convert)

最后，修正处理流程，使得查找资料时，使用改写后的“问题”。

In [10]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)


def log(x):
    print("改写后的问题： " + x)
    return x


retriever_chain = RunnablePassthrough().assign(
    documents=contextualize_chain
    | StrOutputParser()
    | RunnableLambda(log)
    | retriever
    | documents_parser
)

prompt_builder = retriever_chain | prompt_template
prompt = prompt_builder.invoke(
    {
        "question": "英語で回答してください。",
        "history": [
            HumanMessage(content="六元素は何ですか。"),
            AIMessage(content="六元素は日本のIT企業です。"),
        ],
    }
)

show_messages(prompt.to_messages())

改写后的问题： 六元素とは何ですか、英語で説明してください。


In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


def log(x):
    print(x)
    return x


qa = prompt_builder | RunnableLambda(log) | llm

ans = qa.invoke(
    {
        "question": "英語で回答してください。",
        "history": [
            HumanMessage(content="六元素は何ですか。"),
            AIMessage(content="六元素は日本のIT企業です。"),
        ],
    }
)

show_answer(ans)

改写后的问题： Could you please answer in English what the Six Elements is?
messages=[AIMessage(content='What can I help you?'), HumanMessage(content='Please answer my questions using given documents.'), AIMessage(content='OK.'), HumanMessage(content='Sorry. There is no documents found for my question. Please answer my questions without reference documents.'), AIMessage(content='OK. I will answer your questions by original training data.'), HumanMessage(content='六元素は何ですか。'), AIMessage(content='六元素は日本のIT企業です。'), HumanMessage(content='英語で回答してください。')]


In [8]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.base import Runnable
from langchain_core.runnables.history import RunnableWithMessageHistory

# 会话存储位置
global_store = {}


# 根据会话ID，创建或返回既有的历史记录管理部品
def get_session_history(session_id):
    if session_id not in global_store:
        # 历史记录管理使用InMemoryChatMessageHistory
        global_store[session_id] = InMemoryChatMessageHistory()
    return global_store[session_id]


# add a memory to a chain with prompt and chat
qa_with_memory: Runnable = RunnableWithMessageHistory(
    qa,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

res = qa_with_memory.invoke(
    {"question": "六元素は何ですか。"},
    config={"configurable": {"session_id": "abc123"}},
)
show_answer(res)

res = qa_with_memory.invoke(
    {"question": "上記の質問を英語でもう一度回答してください。"},
    config={"configurable": {"session_id": "abc123"}},
)
show_answer(res)

res = qa_with_memory.invoke(
    {"question": "七元素は何ですか。"},
    config={"configurable": {"session_id": "abc123"}},
)
show_answer(res)

Parent run e1c9c0fc-5f25-4f94-a955-6dcabffecaab not found for run 726c6d44-6a3a-473e-884a-6ebbcb791137. Treating as a root run.


改写后的问题： 六番目の元素は何ですか。
messages=[AIMessage(content='What can I help you?'), HumanMessage(content='Please answer my questions using given documents.'), AIMessage(content='OK.'), HumanMessage(content='The documents will send to you one by one.'), AIMessage(content='OK. Please show them one by one.'), HumanMessage(content='[Document 1]\nMetadata: \n- H1: 六元素について\n- score: 3.304002753821667 \n\nContent: \n 「プラットフォーム」になりたい。 私たち六元素は、「ITプロフェッショナル力」による顧客の課題解決と「社会課題」に挑む自社サービス開発で、顧客と社会に感動と幸せを創造する「チャレンジ・プラットフォームカンパニー」です。'), AIMessage(content='OK.'), HumanMessage(content='OK. All 1 documents are sent to you.'), AIMessage(content='Thank you. I will answer your questions based on those 1 documents.'), HumanMessage(content='六元素は何ですか。')]


Parent run bb9bb366-2a81-4faa-8ead-7980be118de3 not found for run 99b290d4-8a4f-4bbe-8655-31d1d9eb7a3f. Treating as a root run.


改写后的问题： Could you please answer the previous question about the six elements in English?
messages=[AIMessage(content='What can I help you?'), HumanMessage(content='Please answer my questions using given documents.'), AIMessage(content='OK.'), HumanMessage(content='Sorry. There is no documents found for my question. Please answer my questions without reference documents.'), AIMessage(content='OK. I will answer your questions by original training data.'), HumanMessage(content='六元素は何ですか。'), AIMessage(content='六元素は、「ITプロフェッショナル力」による顧客の課題解決と「社会課題」に挑む自社サービス開発で、顧客と社会に感動と幸せを創造する「チャレンジ・プラットフォームカンパニー」です。', response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 267, 'total_tokens': 364}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-adeed17c-0e15-4e46-acc9-0abaeba64ef5-0'), HumanMessage(content='上記の質問を英語でもう一度回答してください。')]


Parent run f43c539a-0c1c-4ba9-8005-4d3f6dd48756 not found for run cbf25947-423c-4cac-bbe5-f544f39a3ac0. Treating as a root run.


改写后的问题： 「七元素」は何を指していますか？
messages=[AIMessage(content='What can I help you?'), HumanMessage(content='Please answer my questions using given documents.'), AIMessage(content='OK.'), HumanMessage(content='The documents will send to you one by one.'), AIMessage(content='OK. Please show them one by one.'), HumanMessage(content='[Document 1]\nMetadata: \n- H1: 六元素について\n- H2: 社名由来 ORIGIN OF COMPANY NAME\n- H3: 六元素の意味\n- score: 3.059664541749081 \n\nContent: \n あらゆる生命は活動を維持するために酸素、炭素、窒素、水素、燐、硫黄という六つの元素が必要だとされています。私たちは、企業を発展させるためにも顧客、従業員、株主、社会、パートナー、ライバルという六つの元素が大切だと考えます。さらに中国では、まだ発見されていない未来の元素のことを「第六元素」とも言います。社名にはこれら二つの意味合いから、「堅実な基礎を築き、未来の価値を追求していく」という意味を込めています。'), AIMessage(content='OK.'), HumanMessage(content='OK. All 1 documents are sent to you.'), AIMessage(content='Thank you. I will answer your questions based on those 1 documents.'), HumanMessage(content='六元素は何ですか。'), AIMessage(content='六元素は、「ITプロフェッショナル力」による顧客の課題解決と「社会課題」に挑む自社サービス開発で、顧客と社会に感動と幸せを創造する「チャレンジ・プラットフォームカンパニー」です。', response_me

## 2.简单界面

In [9]:
import ipywidgets as widgets
from IPython.display import clear_output, display
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage

output = widgets.Output(
    layout={
        "height": "300px",
        "padding": "10px",
        "margin": "5px",
        "border-width": "5px",
    }
)

user_input = widgets.Textarea(
    value="",
    placeholder="Enter text here...",
    description="User:",
    rows=2,
    layout=widgets.Layout(width="auto"),
)

# Create button for user to click to send their message
send_button = widgets.Button(description="送信")

conversation_history = []


def send_button_clicked(b):
    # Append user input to conversation history
    conversation_history.append(HumanMessage(content=user_input.value))
    response = qa_with_memory.invoke(
        {"question": user_input.value},
        config={"configurable": {"session_id": "abc123456"}},
    )
    conversation_history.append(response)

    with output:
        clear_output(wait=False)
        show_messages(conversation_history)

    # Clear user input box
    user_input.value = ""


send_button.on_click(send_button_clicked)

# Display everything
display(output, user_input, send_button)

Output(layout=Layout(height='300px', margin='5px', padding='10px'))

Textarea(value='', description='User:', layout=Layout(width='auto'), placeholder='Enter text here...', rows=2)

Button(description='送信', style=ButtonStyle())